<a href="https://colab.research.google.com/github/ahmerrow/DNDB-Scripts/blob/main/DND_PC_Status.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Functions - Run if the bottom section doesn't work.
Runtime - Run All will update the whole page

In [13]:
#@title
import requests, math
import pandas as pd

def get_hp(char_data):
  base_hp = char_data['data']['baseHitPoints'] # purely determined by hit dice
  removed_hp = char_data['data']['removedHitPoints'] # damage
  temp_hp = char_data['data']['temporaryHitPoints']
  bonus_hp_input = char_data['data']['bonusHitPoints']
  if type(bonus_hp_input) is int:
    bonus_hp = bonus_hp_input
  else:
    bonus_hp = 0
  con_score = calc_ability_score(char_data,'CON')
  con_mod = calc_score_modifier(con_score)
  level = get_total_level(char_data)
  max_hp = base_hp + (con_mod * level)
  hp_remaining = max_hp - removed_hp + temp_hp + bonus_hp
  hp_str = str(hp_remaining) + '/' + str(max_hp)
  return hp_str

def get_total_level(char_data):
  level = 0
  class_list = char_data['data']['classes']
  for char_class in class_list:
    level += char_class['level']
  return level



def calc_barbarian_ac(char_data):
  # check if they're wearing armor
  wearing_armor = 0
  for item in char_data['data']['inventory']:
    if item['definition']['filterType'] == 'Armor' and item['equipped'] and item['description']['armorTypeID'] < 4:
      wearing_armor += 1
  # if so calc_standard_ac
  if wearing_armor > 0:
    AC = calc_standard_ac(char_data)
  else:
    # unarmored defense = 10 + DEX + CON
    dex_score = calc_ability_score(char_data,'DEX')
    con_score = calc_ability_score(char_data,'CON')
    dex_mod = calc_score_modifier(dex_score)
    con_mod = calc_score_modifier(con_score)
    AC = 10 + dex_mod + con_mod
  return AC
  
def calc_standard_ac(char_data):
  AC = 0
  dex_score = calc_ability_score(char_data,'DEX')
  dex_mod = calc_score_modifier(dex_score)
  for item in char_data['data']['inventory']:
    if item['equipped'] and item['definition']['filterType'] == 'Armor':
        if item['definition']['armorTypeId'] == 1: # light
          AC += item['definition']['armorClass'] + dex_mod
        elif item['definition']['armorTypeId'] == 2: # medium
          if dex_mod > 2:
            dex_bonus = 2
          else:
            dex_bonus = dex_mod
          AC += item['definition']['armorClass'] + dex_bonus
        elif item['definition']['armorTypeId'] == 3: # heavy
          AC += item['definition']['armorClass']
        elif item['definition']['armorTypeId'] == 4: # shield
          AC += item['definition']['armorClass']
  if AC == 0:
    AC = 10 + dex_mod
  return AC

def get_bonus_ac(char_data):
  bonus_ac = 0
  modifiers = char_data['data']['modifiers']
  for mod in modifiers:
    for entry in modifiers[mod]:
      if entry['subType'] == 'armor-class' and entry['type'] == 'bonus':
        bonus_ac += entry['value']
  return bonus_ac

def get_ac(char_data):
  #need to get class name because of fucking unarmored defense
  #thanksObama
  its_a_fucking_barbarian = 0
  for char_class in char_data['data']['classes']:
    if char_class['definition']['name'] == 'Barbarian':
      its_a_fucking_barbarian += 1
  if its_a_fucking_barbarian > 0:
    AC = calc_barbarian_ac(char_data)
  else:
    AC = calc_standard_ac(char_data)
  AC += get_bonus_ac(char_data)
  return AC

def get_pass_perception(char_data):
  # 10 + perception + proficiency
  wis_score = calc_ability_score(char_data,'WIS')
  wis_modifier = calc_score_modifier(wis_score)
  prof_multiplier = check_proficiency(char_data,'perception')
  prof_bonus = get_proficiency_bonus(char_data)
  proficiency_bonus = prof_multiplier * prof_bonus
  return 10 + wis_modifier + proficiency_bonus

def get_proficiency_bonus(char_data):
  level = get_total_level(char_data)
  if level <= 4:
    prof_bonus = 2
  elif level > 4 and level <= 8:
    prof_bonus = 3
  elif level > 8 and level <= 12:
    prof_bonus = 4
  elif level > 12 and level <= 16:
    prof_bonus = 5
  elif level > 16 and level <= 20:
    prof_bonus = 6
  else:
    prof_bonus = 7
  return prof_bonus

def get_spell_slots(char_data):
  pass #char_data['data']['spellSlots] - display available/used

def check_proficiency(char_data,skill):
  modifiers = char_data['data']['modifiers']
  prof_multiplier = 0
  for mod in modifiers:
    for item in modifiers[mod]:
      if item['subType'] == skill and item['type'] == 'proficiency':
        prof_multiplier = 1
      elif item['subType'] == skill and item['type'] == 'expertise':
        prof_multiplier = 2
  return prof_multiplier

def calc_ability_score(char_data,ability):
  lookup = {
      'STR':[0,'strength-score'],
      'DEX':[1,'dexterity-score'],
      'CON':[2,'constitution-score'],
      'INT':[3,'intelligence-score'],
      'WIS':[4,'wisdom-score'],
      'CHA':[5,'charisma-score']}
  col = lookup[ability]
  score_base = char_data['data']['stats'][col[0]]['value']
  score_bonus_input = char_data['data']['bonusStats'][col[0]]['value']
  if type(score_bonus_input) is int:
    score_bonus = score_bonus_input
  else:
    score_bonus = 0
  modifiers = char_data['data']['modifiers']
  score_modifier = 0
  for mod in modifiers:
    for item in modifiers[mod]:
      if item['subType'] == col[1] and item['type'] == 'bonus':
          score_modifier += item['value']
  return int(score_base + score_modifier + score_bonus)

def calc_score_modifier(score):
  return math.floor((score - 10) / 2) 

def update_players(character_list):
  out_data = {}
  for character in character_list:
    url = 'https://character-service.dndbeyond.com/character/v3/character/' + character_list[character]
    r = requests.get(url)
    out_data[character] = r.json()
  return out_data

def table_output(out_data):
  player_stats = pd.DataFrame(columns = ['Name',
                                        'HP',
                                        'AC',
                                        'PP',
                                        'DEX'])
  for character in out_data:
    name = out_data[character]['data']['name'].split(' ')[0]
    hp = get_hp(out_data[character])
    ac = get_ac(out_data[character])
    pass_perception = get_pass_perception(out_data[character])
    dex = int(calc_ability_score(out_data[character],'DEX'))
    player_stats = player_stats.append({'Name':name,
                                        'HP':hp,
                                        'AC':ac,
                                        'PP':pass_perception,
                                        'DEX':dex},
                                      ignore_index=True)

  player_stats.set_index('Name',inplace=True) 
  print(player_stats)
  return player_stats

# Table Output (Friday Night Campaign)
Click Play next to 'Show Code' to update



In [14]:
#@title
#input 'name':'number' here
#unfortunately i don't have the skills yet to scrape the campaign page to get all these numbers
character_list = {
    'Rhogar':'30223927',
    'Erin  ':'30224048',
    'Haldir':'30635440',
    'Lucius':'30231189',
    'Pants ':'30402453',
    'Casris':'38337729'}

out_data = update_players(character_list)
player_stats = table_output(out_data)

           HP  AC  PP DEX
Name                     
Rhogar  52/52  20  10   8
Erin    33/33  17  17  18
Haldir  32/52  14  15  17
Lucius  44/44  12  11  14
Pants   65/65  15  14  13
Casris  31/31  15  16  15


# Table Output (Sunday Night Campaign)
Click Play next to 'Show Code' to update

In [15]:
#@title
#input 'name':'number' here
#unfortunately i don't have the skills yet to scrape the campaign page to get all these numbers
character_list = {
    'Abed':'41898429',
    'Avery  ':'42117977',
    'Mattrick':'41025428',
    'Thia':'41367244',
    'Thorin ':'41777821',
    'Willow':'42122692'}

out_data = update_players(character_list)
player_stats = table_output(out_data)

             HP  AC  PP DEX
Name                       
Abed,     20/20  18  12   8
Avery       9/9  14  14  17
Mattrick  18/18  15  12  13
Thia      17/17  14  15  15
Thorin    10/10  18  12  12
Willow    12/12  13  12  16
